# Web scraping in Python: Scraping with Requests and BeautifulSoup

Web scraping is a valuable tool for programmers to effortlessly gather information from the vast resources of the internet. While it is generally acceptable for non-commercial purposes with publicly available data, caution should be taken to avoid scraping protected information such as personal data, intellectual property, or confidential information. 

Additionally, the complexities of scraping social media due to its varying levels of accessibility highlight the need for cautious and informed scraping practices.


This coffee and coding session, we will focus to use Python's two libraries; <b>Requests</b> and <b>BeautifulSoup</b>. <br>http://books.toscrape.com/ contains review  for fake books for the beginners learning web scrapings. <br>
This session aims for the beginners (like me!) introduction to web scraping.

To gather information from the internet through web scraping, one typically follows a four-step process:

<li> Sending an HTTP GET request to the URL </li>
<li> Retrieving HTML (Hypertext Markup Language) content </li>
<li> Building the HTML document tree </li>
<li> Extracting information from the HTML document tree </li>   

### Requests


The Requests library in Python is a popular and widely used library for making HTTP requests. This library allows user to send HTTP requests to server, receive response and handle in a simple and efficient manner. 

It supports varous methods for making requests, such as GET, POST, HEAD, PUT, DELETE etc.

### BeautifulSoup

BeautifulSoup is a Python library for web scraping and data extraction from HTML and XML files. <br>
It provides a convenient and efficient way to parse and naviage through HTML contents, allowing for the extraction of specific elements and data.

<b>Resource: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ </b> 

We will first fetch HTML code from our fake-books site. But before we start, I will show you a very quick overview of basic HTML. 

HTML is a standard markup language used for creating web pages and other information that can be displayed in a web browser. It consists of a set of tags and attributes that define the structure, content, and appearance of a web page. 

In [ ]:
from IPython.core.display import HTML

In [ ]:
html_test = """
<html>
  <head>
    <title>My First Web Page</title>
  </head>
  <body>
    <h1>Welcome to my first test web page</h1>
    <p>This is a paragraph of text. I want to list some of my favourite composers and their music 
        <i> Italic font </i>    
        <b> Bold text 1</b>
        <b> Bold text 2</b>
    </p>    
    <p> <b color = "blue"> This is next paragraph </b></p>
    <br>    
    <ul id = "composer" class = "myclass">
      <li>Liszt</li>  
      <a href = "https://en.wikipedia.org/wiki/Franz_Liszt"> Franz Liszt: Hungarian composer, pianist of romantic period. </a>
      <li>Mozart</li>
      <a href = "https://en.wikipedia.org/wiki/Wolfgang_Amadeus_Mozart"> Wolfgang Amadeus Mozart influential composer of classical period. </a>
      <li>Debussy</li>
      <a href = "https://en.wikipedia.org/wiki/Claude_Debussy"> Claude Debussy, French composer seen as the first impressionist. </a>
      <li>Beethoven</li>
      <a href = "https://en.wikipedia.org/wiki/Ludwig_van_Beethoven"> Ludwig van Beethoven, German composer and pianist.</a>
    </ul>
    <ul id = "piece" class = "myclass">
        <li>Love Dream (No.3)</li>
        <li>Piano Sonata No.16</li>
        <li>Moonlight</li>
        <li>Symphony No.5</li>
    </ul>
  </body>
</html>
"""

In [ ]:
# Let's import libraries first 
from bs4 import BeautifulSoup

We explore functions in BeautifulSoup using the sample HTML we've just created. 

In [ ]:
# create soup variable
soup = BeautifulSoup(html_test, features = "html.parser")

In [ ]:
# soup has the information extracted from HTML string. We can make it better display
soup

In [ ]:
# Now indent etc works better
# Prettify arranges all the tags in a parse-tree manner with better readability.
print(soup.prettify())

When you read HTML, you want to search specific aspect. You can find thins by `tag`. like `<head>`,`<title>` etc. 

In [ ]:
# Codes to nativage data structure (https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
display(soup.title) # access title tag
display(soup.title.string) # acces title tag and want only access what is inside in the tag
# You can also modify string in your tag
# soup.title.string = "My Second web page"
display(soup.p.get_text())
display(soup.ul.get_text()) # it returns first ul element only.

Let's try `find()` or `find_all()` functions to search for specific tags in the HTML content. <br>
`find()` returns only the first occurrence of the search query. `find_all()` returns a list of all matches.

In [ ]:
display(soup.find('a')) # first element of a tag only
display(soup.find('a').text) # This allows us to extract the inner HTML text
display(soup.find_all('a')) # all a tag elements

In [ ]:
display(soup.find_all("p")[0]) # First element of p tag
display(soup.find_all("p")[0].find_all("b")) # inside p tag all b tags

If we only want to extract composer items `<li>`, we can use `attr ={}` dictionary to define the attributes of an HTML tag. Dictionary keys are the name of the attributes, and the values are the attribute values.<br>

In [ ]:
myList = soup.find(attrs = {'id':'composer', 'class':'myclass'})
myList.find_all('li')

You can also traverse the parent and children elements in the HTML code. Below code uses `find_parent()` function to find the parent of the first `ul` tag (`body` tag). It then uses `find_all()` function again to find all the `li` elements in the first `ul` tag and print each `li` element.

In [ ]:
# First find the first ul tag
first_ul = soup.find("ul")

# Find the parent of the first ul tag (the body tag)
body = first_ul.find_parent("body")

# Print the text content of each li element
for li in first_ul.find_all(["a", "li"]):
    print(li.text)


### Let's try with fake-book review site 

We've tried few basics of beautifulsoup functions using our own HTML text. Now, we can go to fake book review sites and extract some information about the books, their prices and book description.
Before we start, let's check this webpage and inspect HTML codes. https://books.toscrape.com/catalogue/page-1.html

In [323]:
# Import requests and pandas, re as well for regex string manipulation 
import requests
import pandas as pd
import re
from tqdm import tqdm

#### Read the first page



First, read book title, price and rating. We will then extend this by adding book description. (When user click book title, it takes to the product description page)

In [ ]:
# request can get HTML data from the very first page.
url = "https://books.toscrape.com/catalogue/page-1.html" #page-2 etc will repeat to extract all pages. which we will try in the next section
response = requests.get(url) # http request and get the content of the page
display(response)
# 200 means 'Success' status
# If you want to see content of resonse (it will be messy)
# response.text

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
# You can look at the content using 
# soup.prettify()

Let's go back to our html inspect page extract our interest (book title, price and later description).

In [ ]:
# Extract first book in the first page to see if this is working.
review_mapping = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5} # For mapping for numbers

article = soup.find("article")
title = article.h3.a["title"]
print(title)
price = article.find('p', class_="price_color").text 
price = float(re.findall("\d+\.\d+", price)[0]) #regex + is metacharacters means one or more occurrences
print(price)
rating = article.p["class"][1] # access secound element one, two, three etc 
rating = review_mapping[rating]
print(rating)

In [330]:
# Based on the code above, we can now extract all books in the first page.
books = [] # We will append to this list, define empty list

for article in soup.find_all("article"):
    title = article.h3.a["title"]
    # price = article.select_one(".price_color").get_text() # another way using select_one (https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors)    
    price = article.find('p', class_="price_color").text 
    price = float(re.findall("\d+\.\d+", price)[0])
    rating = article.p["class"][1] # access secound element one, two etc 
    rating = review_mapping[rating]
    books.append({"title": title, "price": price, "rating":rating})
      
for book in books[0:5]: # First 5 books
    print(f"Title: {book['title']}")
    print(f"Price: {book['price']}")
    print(f"Rating: {book['rating']}")
    print("")   


Title: Frankenstein
Price: 38.0
Rating: 2

Title: Forever Rockers (The Rocker #12)
Price: 28.8
Rating: 3

Title: Fighting Fate (Fighting #6)
Price: 39.24
Rating: 3

Title: Emma
Price: 32.93
Rating: 2

Title: Eat, Pray, Love
Price: 51.32
Rating: 3



In [338]:
# Expand the code from above, include book description
# import requests
# from bs4 import BeautifulSoup

# # request the main page
# url = "https://books.toscrape.com/catalogue/page-1.html" #page-2 etc will repeat to extract all pages for next step. 
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")

# # Find all book titles, prices, and customer reviews
books = []
review_mapping = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5} # For mapping for numbers

for article in soup.find_all("article"):
    title = article.h3.a["title"]
    price = article.find('p', class_="price_color").text 
    price = float(re.findall("\d+\.\d+", price)[0])
    rating = article.p["class"][1] # access secound element one, two etc 
    rating = review_mapping[rating]
    
    # find the link to the individual book's page
    link = article.h3.a["href"]
    split_link = link.split("/")
    # link: a-light-in-the-attic_1000/index.html etc 
    # print(link) 
    
    book_url = "http://books.toscrape.com/catalogue/" + link  #get information of individual book link page to take you to the description  
    
    # request the individual book's page
    book_response = requests.get(book_url) #request again to get text of the individual book page.
    book_soup = BeautifulSoup(book_response.text, "html.parser")
    
    # extract the product description
    # select a meta tag with attribute name = "description". The method select_one returns the first matching element, or 'None'
    # if there are not matches. ['content'] part is accessing the value of the content attribute of the selected meta tag. 
    
    product_description = book_soup.select_one("meta[name='description']")["content"]  
    books.append({"title": title, "price": price, "rating":rating, "book_description": product_description})
    
    

for book in books[0:5]: # First 5 books
    print(f"Title: {book['title']}")
    print(f"Price: {book['price']}")
    print(f"Rating: {book['rating']}")
    print(f"Description: {book['book_description']}")
    print("")
    

Title: Frankenstein
Price: 38.0
Rating: 2
Description: 
    Mary Shelley began writing Frankenstein when she was only eighteen. At once a Gothic thriller, a passionate romance, and a cautionary tale about the dangers of science, Frankenstein tells the story of committed science student Victor Frankenstein. Obsessed with discovering the cause of generation and life and bestowing animation upon lifeless matter, Frankenstein assembles Mary Shelley began writing Frankenstein when she was only eighteen. At once a Gothic thriller, a passionate romance, and a cautionary tale about the dangers of science, Frankenstein tells the story of committed science student Victor Frankenstein. Obsessed with discovering the cause of generation and life and bestowing animation upon lifeless matter, Frankenstein assembles a human being from stolen body parts but; upon bringing it to life, he recoils in horror at the creature's hideousness. Tormented by isolation and loneliness, the once-innocent creature tu

In [341]:
# Final stage: Expand further, scraping ALL pages. 
# However, it takes too long so we will not run this in the session.
# I have saved as csv file.
# I will explain how I get final page (50) from tag

# %%time
review_mapping = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
books = []

# # Get the total number of pages
url = "http://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

previous_page = soup.select("li.next")[0].find_previous_sibling("li")
if previous_page:
    text = previous_page.text # which returns page 1 of 50 and we want to extract 50 from here.
    print(text)
    # in here extract the last page number 
    match = re.search(r'Page (\d+) of (\d+)', text) # any number
    total_pages = int(match.group(2)) # extract last page.
    print(total_pages)
else:
    total_pages = 1


# Loop through each page

# for page_number in tqdm(range(1, total_pages + 1)):
#     url = f"http://books.toscrape.com/catalogue/page-{page_number}.html"
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     for article in soup.find_all("article"):
#         title = article.h3.a["title"]
#         price = article.find("p", class_="price_color").text 
#         price = float(re.findall("\d+\.\d+", price)[0])
#         rating = article.p["class"][1]
#         rating = review_mapping[rating]
#         link = article.h3.a["href"]
#         book_url = "http://books.toscrape.com/catalogue/" + link
#         book_response = requests.get(book_url)
#         book_soup = BeautifulSoup(book_response.text, "html.parser")
#         product_description = book_soup.select_one("meta[name='description']")["content"]  
#         books.append({"title": title, "price": price, "product_description": product_description})
        
# # Convert the list of dictionaries to a Pandas dataframe
# # We will clean the text
# df = pd.DataFrame(books)

# df.head()
# We will use this for the next.
# df.to_csv("books_web_scraping.csv", index = False)



            
                Page 1 of 50
            
            
50
